# Transform

Testing ground for service call data transformation

In [1]:
from pathlib import Path
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

In [2]:
spark = SparkSession.builder.master("local[*]").appName("test").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/19 10:10:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
pq_path = Path("../tests/resources/SR2020.parquet")
df = spark.read.parquet(str(pq_path))
df.printSchema()

root
 |-- Status: string (nullable = true)
 |-- First 3 Chars of Postal Code: string (nullable = true)
 |-- Intersection Street 1: string (nullable = true)
 |-- Intersection Street 2: string (nullable = true)
 |-- Service Request Type: string (nullable = true)
 |-- Division: string (nullable = true)
 |-- Section: string (nullable = true)
 |-- ward_name: string (nullable = true)
 |-- ward_id: byte (nullable = true)
 |-- creation_datetime: timestamp (nullable = true)



In [ ]:
# stop current SparkContext before creating a new one
spark.stop()

Read from cloud storage using `gcs-connector-hadoop3` jar

Before starting a new context, I had to restart the kernel for spark to recognize the gcs-connector jar

In [31]:
!mkdir ../data/lib
!gsutil cp gs://hadoop-lib/gcs/gcs-connector-hadoop3-latest.jar  \
    ../data/lib/gcs-connector-hadoop3-latest.jar  
# !curl -O https://github.com/GoogleCloudDataproc/hadoop-connectors/releases/download/v2.2.11/gcs-connector-hadoop3-2.2.11-shaded.jar
    

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0


In [2]:
conf = (
    SparkConf()
    .setMaster("local[*]")
    .setAppName("test_cloud")
    .set("spark.jars", "../data/lib/gcs-connector-hadoop3-latest.jar")
)
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set(
    "fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS"
)
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
spark = SparkSession.builder.config(conf=sc.getConf()).getOrCreate()

23/03/19 10:55:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
import os
from dotenv import load_dotenv

In [5]:
load_dotenv()
DATA_LAKE = os.getenv("DATA_LAKE")
gcs_path = f"gs://{DATA_LAKE}/raw/pq/sr2023.parquet"
print(f"gcs_path: {gcs_path}")
df_gcs = spark.read.parquet(gcs_path)
df_gcs.printSchema()

gcs_path: gs://service-calls-data-lake/raw/pq/sr2023.parquet


root
 |-- Status: string (nullable = true)
 |-- First 3 Chars of Postal Code: string (nullable = true)
 |-- Intersection Street 1: string (nullable = true)
 |-- Intersection Street 2: string (nullable = true)
 |-- Service Request Type: string (nullable = true)
 |-- Division: string (nullable = true)
 |-- Section: string (nullable = true)
 |-- ward_name: string (nullable = true)
 |-- ward_id: byte (nullable = true)
 |-- creation_datetime: timestamp (nullable = true)

